# Prepare

In [1]:
import awkward as ak
import numpy as np
import os, json, time, uproot, yaml, correctionlib

from coffea import processor, lumi_tools
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.nanoevents.methods.nanoaod import FatJetArray
from coffea.analysis_tools import PackedSelection
from coffea.lookup_tools import extractor
from coffea.jetmet_tools import JECStack, CorrectedJetsFactory, CorrectedMETFactory

import sys
sys.path.append("..")
from src.processors.Processor import Processor
from src.processors.TriggerProcessor import TriggerProcessor
from src.processors.GenMatch import GenMatch
os.chdir(os.path.join(os.getcwd(), 'postprocess'))
#import importlib
#importlib.reload(Processor)

/home/pku/fudawei/anaconda3/lib/python3.9/site-packages/coffea/nanoevents/schemas/fcc.py:5: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector


## Correction

In [2]:
import awkward as ak
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.lookup_tools import extractor

fname = "~/nano_dy.root"
events = NanoEventsFactory.from_root(
    {fname: 'Events'},
    schemaclass=NanoAODSchema,
    metadata={"dataset": "DYJets"}, delayed=False
).events()

FileNotFoundError: [Errno 2] No such file or directory: '/home/pku/fudawei/nano_dy.root'

In [ ]:
#%%bash
# download some sample correction sources
mkdir -p data
pushd data
PREFIX=https://raw.githubusercontent.com/CoffeaTeam/coffea/master/tests/samples
curl -Os $PREFIX/testSF2d.histo.root
curl -Os $PREFIX/Fall17_17Nov2017_V32_MC_L2Relative_AK4PFPuppi.jec.txt
curl -Os $PREFIX/Fall17_17Nov2017_V32_MC_Uncertainty_AK4PFPuppi.junc.txt
curl -Os $PREFIX/DeepCSV_102XSF_V1.btag.csv.gz
popd

In [ ]:
ext = extractor()
# several histograms can be imported at once using wildcards (*)
ext.add_weight_sets(["testSF2d scalefactors_Tight_Electron data/testSF2d.histo.root"])
ext.finalize()

evaluator = ext.make_evaluator()

print("available evaluator keys:")
for key in evaluator.keys():
    print("\t", key)
print("testSF2d:", evaluator['testSF2d'])
print("type of testSF2d:", type(evaluator['testSF2d']))

In [ ]:
print("Electron eta:", events.Electron.eta)
print("Electron pt:", events.Electron.pt)
print("Scale factor:", evaluator["testSF2d"](events.Electron.eta, events.Electron.pt))

## CMS high-level tools

In [ ]:
from coffea.jetmet_tools import FactorizedJetCorrector, JetCorrectionUncertainty
from coffea.jetmet_tools import JECStack, CorrectedJetsFactory
import awkward as ak
import numpy as np

ext = extractor()
ext.add_weight_sets([
    "* * data/Fall17_17Nov2017_V32_MC_L2Relative_AK4PFPuppi.jec.txt",
    "* * data/Fall17_17Nov2017_V32_MC_Uncertainty_AK4PFPuppi.junc.txt",
])
ext.finalize()

jec_stack_names = [
    "Fall17_17Nov2017_V32_MC_L2Relative_AK4PFPuppi",
    "Fall17_17Nov2017_V32_MC_Uncertainty_AK4PFPuppi"
]

evaluator = ext.make_evaluator()

jec_inputs = {name: evaluator[name] for name in jec_stack_names}
jec_stack = JECStack(jec_inputs)
### more possibilities are available if you send in more pieces of the JEC stack
# mc2016_ak8_jxform = JECStack(["more", "names", "of", "JEC parts"])

print(dir(evaluator))

In [ ]:
[i for i in corrected_jets.fields if i.startswith("JES") or i.startswith("JER")]

In [ ]:
print('untransformed pt ratios', jets.pt/jets.pt_raw)
print('untransformed mass ratios', jets.mass/jets.mass_raw)

print('transformed pt ratios', corrected_jets.pt/corrected_jets.pt_raw)
print('transformed mass ratios', corrected_jets.mass/corrected_jets.mass_raw)

print('JES UP pt ratio', corrected_jets.JES_jes.up.pt/corrected_jets.pt_raw)
print('JES DOWN pt ratio', corrected_jets.JES_jes.down.pt/corrected_jets.pt_raw)

# Batch test

In [2]:
uproot.open.defaults["xrootd_handler"] = uproot.MultithreadedXRootDSource
_events = NanoEventsFactory.from_root(
    #{'/data/bond/fudawei/public/customizedNanoAOD/output/mc/2017/GJets/GJets_HT-600ToInf/209.root': 'Events'},
    {'~/custom_nano.root': 'Events'},
    #{'/data/bond/fudawei/public/customizedNanoAOD/output/mc/2017/TTJets/TTJets/2339.root': 'Events'},
    #{'/data/bond/fudawei/public/customizedNanoAOD/output/mc/2017/ZpToHG/ZpToHGamma_M1000/14.root': 'Events'},
    schemaclass=NanoAODSchema, delayed=False,
).events()
p=Processor(outdir='./test/', mode='mc_2018_ZpToHG', param_dir='../src/parameters/')
c=p.process(_events)

/home/pku/fudawei/anaconda3/lib/python3.9/site-packages/coffea/nanoevents/schemas/nanoaod.py:264: RuntimeWarning: Missing cross-reference index for LowPtElectron_electronIdx => Electron
  warnings.warn(
/home/pku/fudawei/anaconda3/lib/python3.9/site-packages/coffea/nanoevents/schemas/nanoaod.py:264: RuntimeWarning: Missing cross-reference index for LowPtElectron_photonIdx => Photon
  warnings.warn(


In [11]:
import awkward as ak
import numpy as np
import os, json, time, uproot, yaml, correctionlib

from coffea import processor, lumi_tools
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.nanoevents.methods.nanoaod import FatJetArray
from coffea.analysis_tools import PackedSelection
from coffea.lookup_tools import extractor
from coffea.jetmet_tools import JECStack, CorrectedJetsFactory, CorrectedMETFactory


_events = NanoEventsFactory.from_root(
    #{'/data/bond/fudawei/public/customizedNanoAOD/output/mc/2017/GJets/GJets_HT-600ToInf/209.root': 'Events'},
    {'~/custom_nano.root': 'Events'},
    #{'/data/bond/fudawei/public/customizedNanoAOD/output/mc/2017/TTJets/TTJets/2339.root': 'Events'},
    #{'/data/bond/fudawei/public/customizedNanoAOD/output/mc/2017/ZpToHG/ZpToHGamma_M1000/14.root': 'Events'},
    schemaclass=NanoAODSchema, delayed=False,
).events()

AK8jet = _events.FatJet

extract = extractor()
uncertainty_dir = os.path.join('src/parameters', 'uncertainty', '2018')
for f in os.listdir(uncertainty_dir):
    if 'AK8' not in f or 'UncertaintySources' in f:
        continue
    extract.add_weight_sets([f'* * {os.path.join(uncertainty_dir, f)}'])

extract.finalize()
evaluator = extract.make_evaluator()

jec_names = dir(evaluator)
jec_inputs = {name: evaluator[name] for name in jec_names}
jec_stack = JECStack(jec_inputs)
name_map = jec_stack.blank_name_map
name_map['JetPt'] = 'pt'
name_map['JetMass'] = 'mass'
name_map['JetEta'] = 'eta'
name_map['JetPhi'] = 'phi'
name_map['JetA'] = 'area'
name_map['ptGenJet'] = 'pt_gen'
name_map['ptRaw'] = 'pt_raw'
name_map['massRaw'] = 'mass_raw'
name_map['Rho'] = 'PU_rho'

AK8jet['pt_original'] = AK8jet.pt
AK8jet['is_real'] = (~np.isnan(ak.fill_none(AK8jet.matched_gen.pt, np.nan)))*1
AK8jet["pt_raw"] = (1 - AK8jet.rawFactor) * AK8jet.pt
AK8jet["mass_raw"] = (1 - AK8jet.rawFactor) * AK8jet.mass
AK8jet["pt_gen"] = ak.values_astype(ak.fill_none(AK8jet.matched_gen.pt, 0), np.float32)
AK8jet["PU_rho"] = ak.broadcast_arrays(_events.fixedGridRhoFastjetAll, AK8jet.pt)[0]
corrected_AK8jet = CorrectedJetsFactory(name_map, jec_stack).build(AK8jet).compute()
AK8jet["pt_nominal"] = corrected_AK8jet["pt"]
AK8jet["mass_nominal"] = corrected_AK8jet["mass"]

AK8jet_corrections = set()
for i in corrected_AK8jet.fields:
    if i.startswith("JES") or i.startswith("JER"):
        AK8jet_corrections.add(i)
        AK8jet[f"pt_{i}_up"] = corrected_AK8jet[i].up.pt
        AK8jet[f"pt_{i}_down"] = corrected_AK8jet[i].down.pt
        AK8jet[f"mass_{i}_up"] = corrected_AK8jet[i].up.mass
        AK8jet[f"mass_{i}_down"] = corrected_AK8jet[i].down.mass

/home/pku/fudawei/.local/lib/python3.9/site-packages/coffea/nanoevents/schemas/nanoaod.py:247: RuntimeWarning: Missing cross-reference index for LowPtElectron_electronIdx => Electron
  warnings.warn(
/home/pku/fudawei/.local/lib/python3.9/site-packages/coffea/nanoevents/schemas/nanoaod.py:247: RuntimeWarning: Missing cross-reference index for LowPtElectron_photonIdx => Photon
  warnings.warn(


In [15]:
_events.FatJet.pt - AK8jet['pt']

<Array [[0, 0], [0, 0], ..., [0, ...], [0, 0]] type='939 * var * float32[pa...'>